In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date, timedelta

In [2]:
df = pd.read_csv('../Datasets/Covid/covid_19_clean_complete.csv')

In [3]:
print(df.info())  ##General info for dataset
df[df['Country/Region']=='Greece'].tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15064 entries, 0 to 15063
Data columns (total 8 columns):
Province/State    7056 non-null object
Country/Region    15064 non-null object
Lat               15064 non-null float64
Long              15064 non-null float64
Date              15064 non-null object
Confirmed         15064 non-null int64
Deaths            15064 non-null int64
Recovered         15064 non-null int64
dtypes: float64(2), int64(3), object(3)
memory usage: 941.6+ KB
None


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
13756,NaN,Greece,39.0742,21.8243,3/13/20,190,1,0
14025,NaN,Greece,39.0742,21.8243,3/14/20,228,3,8
14294,NaN,Greece,39.0742,21.8243,3/15/20,331,4,8
14563,NaN,Greece,39.0742,21.8243,3/16/20,331,4,8
14832,NaN,Greece,39.0742,21.8243,3/17/20,387,5,8


In [4]:
df['Day']=df['Date'].apply(lambda x: int(x.split('/')[1]))           ##Turn Date into
df['Month']=df['Date'].apply(lambda x: int(x.split('/')[0]))         ##usefull, countable
df.drop(['Date'],axis=1,inplace=True)                                ##data.

In [5]:
Country_List = df['Country/Region'].unique() #List of all countries in dataset
Confirmed_List = [] #List of confirmed cases of the countries in Country_List
Deaths_List = [] #List of deaths in above of the countries in Country_List
D_Confirmed_List = [] #List of change in the number of confirmed cases of the countries in Country_List
D_Deaths_List = [] #List of change in the number of deaths of the countries in Country_List

dt = 1 # How many days before today to be set as the reference point for the collected data

for i in Country_List:
    Confirmed_List.append(df[df['Country/Region']==i][df['Month']==int(str(date.today()-timedelta(days=dt)).split('-')[1])][df['Day']==int(str(date.today()-timedelta(days=dt)).split('-')[2])]['Confirmed'].to_numpy()[0])
    Deaths_List.append(df[df['Country/Region']==i][df['Month']==int(str(date.today()-timedelta(days=dt)).split('-')[1])][df['Day']==int(str(date.today()-timedelta(days=dt)).split('-')[2])]['Deaths'].to_numpy()[0])
    D_Confirmed_List.append(df[df['Country/Region']==i][df['Month']==int(str(date.today()-timedelta(days=dt)).split('-')[1])][df['Day']==int(str(date.today()-timedelta(days=dt)).split('-')[2])]['Confirmed'].to_numpy()[0]-df[df['Country/Region']==i][df['Month']==int(str(date.today()-timedelta(days=dt+1)).split('-')[1])][df['Day']==int(str(date.today()-timedelta(days=dt+1)).split('-')[2])]['Confirmed'].to_numpy()[0])
    D_Deaths_List.append(df[df['Country/Region']==i][df['Month']==int(str(date.today()-timedelta(days=dt)).split('-')[1])][df['Day']==int(str(date.today()-timedelta(days=dt)).split('-')[2])]['Deaths'].to_numpy()[0]-df[df['Country/Region']==i][df['Month']==int(str(date.today()-timedelta(days=dt+1)).split('-')[1])][df['Day']==int(str(date.today()-timedelta(days=dt+1)).split('-')[2])]['Deaths'].to_numpy()[0])
    
Sorted_Confirmed_List = np.argsort(Confirmed_List)  #Sort by Confirmed Cases
Sorted_Deaths_List = np.argsort(Deaths_List)  # Sort by Deaths
Sorted_D_Confirmed_List = np.argsort(D_Confirmed_List) #Sort by change in Confirmed Cases
Sorted_D_Deaths_List = np.argsort(D_Deaths_List) #Sort by change in Deaths

c:\users\alexa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
c:\users\alexa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
c:\users\alexa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
c:\users\alexa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


In [6]:
for i in np.flip(Sorted_Confirmed_List[-3:]): #Printing the stats on the 4 countries with most confirmed cases
    print(Country_List[i]+
          ' \nConfirmed:',Confirmed_List[i],
          ' \nonly from the previous day:',D_Confirmed_List[i],
          ' \nDeaths:',Deaths_List[i],
          ' \nonly from the previous day:',D_Deaths_List[i],
          ' \n')

print('As of '+str(date.today()-timedelta(days=dt))+'.')

China 
Confirmed: 67799  
only from the previous day: 1  
Deaths: 3111  
only from the previous day: 12  

Italy 
Confirmed: 31506  
only from the previous day: 3526  
Deaths: 2503  
only from the previous day: 345  

Iran 
Confirmed: 16169  
only from the previous day: 1178  
Deaths: 988  
only from the previous day: 135  

As of 2020-03-17.


In [7]:
for i in np.flip(Sorted_D_Confirmed_List[-3:]): #Printing the stats on the 4 countries with biggest change of confirmed cases
    print(Country_List[i]+
          ' \nConfirmed:',Confirmed_List[i],
          ' \nonly from the previous day:',D_Confirmed_List[i],
          ' \nDeaths:',Deaths_List[i],
          ' \nonly from the previous day:',D_Deaths_List[i],
          ' \n')

print('As of '+str(date.today()-timedelta(days=dt))+'.')

Italy 
Confirmed: 31506  
only from the previous day: 3526  
Deaths: 2503  
only from the previous day: 345  

Germany 
Confirmed: 9257  
only from the previous day: 1985  
Deaths: 24  
only from the previous day: 7  

Spain 
Confirmed: 11748  
only from the previous day: 1806  
Deaths: 533  
only from the previous day: 191  

As of 2020-03-17.
